### **1. Importanto as bibliotecas necessárias**

In [ ]:
import tensorflow as tf
import pandas as pd

from sklearn.model_selection import train_test_split
from utils.utils import split_data

from crack_segmentation.subroutines.loss_metrics import (
    Weighted_Cross_Entropy,
    F1_score,
    Precision_dil,
    F1_score_dil
)

### **2. Carreganto os dados**

In [ ]:
dataset = pd.read_csv('../dataset/dataset_final.csv')

dataset['Filepath'] = dataset['Filepath'].apply(lambda x: "../" + x)

train_df, test_df = train_test_split(
    dataset.sample(6000, random_state=42),
    train_size=0.80,
    random_state=42
)

_,_, test_data = split_data(train_df, test_df, image_width=224, image_height=224, class_mode='categorical')

### **3. Modelo de segmentação e classificação**

#### **3.1. Importando o modelo de classificação**

In [ ]:
model = tf.keras.models.load_model(
    'crack_segmentation/output/checkpoints/final_model_2.h5',
    custom_objects={
        'loss': Weighted_Cross_Entropy(10),
        'F1_score': F1_score,
        'F1_score_dil': F1_score_dil,
        'Precision_dil': Precision_dil
    }
)

model.summary()

#### **3.2. Fazendo predições**

In [ ]:
loss, acc = model.evaluate(test_data)

print(f'Perda: {loss}')
print(f'Acurácia: {acc}')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

pred = model.predict(test_data)